# Ridge

In [1]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

## Get Data

In [2]:
from common.data import get_data

X, Y, demographics, population = get_data(wisc_level=5)
ages = demographics['Age']

print(f'X: {X.shape} | Y: {len(Y.keys())} | Age: {ages.shape} | Population: {population}')

X: (390, 34716) | Y: 6 | Age: (390,) | Population: adhd


In [3]:
from scipy import stats
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import RepeatedKFold, cross_validate, permutation_test_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

## Run for one target, one age bin

In [90]:
selected_target = "WISC_FSIQ"
y = Y[selected_target]

print(f'{selected_target}: {y.shape}')

WISC_FSIQ: (390,)


In [74]:
from common.wisc import WISC_LEVEL

y = np.array([Y[measure] for measure in WISC_LEVEL[5]]).T
print(f'{selected_target}: {y.shape}')

WISC_FSIQ: (390, 6)


In [91]:
from common.binning import bin_by_age

bins = bin_by_age(X, y, ages, y)
bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
print(f'Bin 1: {bin_1[0].shape} | Bin 2: {bin_2[0].shape} | Bin 3: {bin_3[0].shape}')

Bin 1: (117, 34716) | Bin 2: (156, 34716) | Bin 3: (117, 34716)


### Set up the model pipeline, metrics, and cross-validation approach

In [79]:
def regression_scorer(reg, X, y):
    y_pred = reg.predict(X)
    return stats.pearsonr(y, y_pred)[0]

scoring = ['train_score', 'test_score']
# rkf = RepeatedKFold(n_splits=10, n_repeats=10)
rkf = RepeatedKFold(n_splits=10, n_repeats=1)
n_perm = 3000

In [86]:
%%time
# X_cv = bin_1[0]
# y_cv = bin_1[1]
X_cv = X
y_cv = y
age_group = 'all'

# [a for a in range(5000, 55000, 5000)]
estimators = [StandardScaler(), MultiOutputRegressor(
    RidgeCV(alphas=[5000], scoring=regression_scorer, cv=rkf))]
pipe = make_pipeline(*estimators)
pipe.fit(X_cv, y_cv)
ridge_cvs = pipe['multioutputregressor'].estimators_

for target, ridge_cv in zip(WISC_LEVEL[5], ridge_cvs):
    print(f'Target: {target} | Alpha: {ridge_cv.alpha_} | Score: {ridge_cv.best_score_:.2f}')

Target: WISC_FSIQ | Alpha: 5000 | Score: 0.38
Target: WISC_VSI | Alpha: 5000 | Score: 0.25
Target: WISC_VCI | Alpha: 5000 | Score: 0.39
Target: WISC_FRI | Alpha: 5000 | Score: 0.26
Target: WISC_WMI | Alpha: 5000 | Score: 0.20
Target: WISC_PSI | Alpha: 5000 | Score: 0.10
CPU times: user 36.5 s, sys: 1.32 s, total: 37.8 s
Wall time: 4.01 s


### Run cross-validation

In [37]:
%%time
estimators = [StandardScaler(), Ridge(alpha=)]
pipe = make_pipeline(*estimators)

scores = cross_validate(pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_jobs=-1, 
                        return_train_score=True, return_estimator=True)

coefs = np.array([estimator['ridge'].coef_ for estimator in scores['estimator']])
avg_coef = np.mean(coefs, axis=0)
intercepts = np.array([estimator['ridge'].intercept_ for estimator in scores['estimator']])
avg_inte = np.mean(intercepts, axis=0)

print(f'ridge_{population}_{selected_target}_{age_group}')
for metric in scoring:
    metric_values = scores[metric]
    print(f'Avg {metric}: {np.mean(metric_values):.3f}')

ridge_healthy_WISC_FSIQ_all
Avg train_score: 0.996
Avg test_score: 0.129
CPU times: user 2.33 s, sys: 333 ms, total: 2.66 s
Wall time: 2.72 s


In [38]:
from os.path import join
from common.paths import RIDGE_WEIGHTS

coef_fn = f'ridge_{population}_{selected_target}_{age_group}_coef.npy'
inte_fn = f'ridge_{population}_{selected_target}_{age_group}_inte.npy'

np.save(join(RIDGE_WEIGHTS, coef_fn), avg_coef)
np.save(join(RIDGE_WEIGHTS, inte_fn), avg_inte)

### Transfer model weights

In [ ]:
from os.path import join
from common.paths import RIDGE_WEIGHTS

weight_pop = 'adhd'
weight_tar = 'WISC_FSIQ'
weight_age = 'all'
coef_fn = f'ridge_{weight_pop}_{weight_tar}_{weight_age}_coef.npy'
inte_fn = f'ridge_{weight_pop}_{weight_tar}_{weight_age}_inte.npy'

alpha = 5000
intercept = 108.534744

ridge = Ridge(alpha=alpha)
ridge.coef_ = np.load(join(RIDGE_WEIGHTS, coef_fn))
ridge.intercept_ = np.load(join(RIDGE_WEIGHTS, inte_fn))

print('Score:', regression_scorer(ridge, X, y))

#### ICC

In [34]:
%%time
import pingouin as pg

icc_data = pd.DataFrame(coefs).melt(var_name='connection', value_name='weight', ignore_index=False)
icc_data['cv_run_num'] = icc_data.index
icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
icc.set_index("Type")

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.897,873.76,34715,3436884,0.0,"[0.9, 0.9]"
ICC2,Single random raters,0.897,873.88,34715,3436785,0.0,"[0.9, 0.9]"
ICC3,Single fixed raters,0.897,873.88,34715,3436785,0.0,"[0.9, 0.9]"
ICC1k,Average raters absolute,0.999,873.76,34715,3436884,0.0,"[1.0, 1.0]"
ICC2k,Average random raters,0.999,873.88,34715,3436785,0.0,"[1.0, 1.0]"
ICC3k,Average fixed raters,0.999,873.88,34715,3436785,0.0,"[1.0, 1.0]"


### Run permutation statistic

In [37]:
%%time
score, _, pvalue = permutation_test_score(
    pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_permutations=n_perm, n_jobs=-1)

print(f'ridge_{population}_{selected_target}_{age_group}')
print(f'Alpha: {ridge_cv.alpha_} | Score: {score:.2f} | p-value: {pvalue:.4f}')

ridge_healthy_WISC_PSI_all
Alpha: 5000 | Score: -0.04 | p-value: 0.6268
CPU times: user 1min 13s, sys: 2.12 s, total: 1min 15s
Wall time: 1h 5min 49s


## Run for all targets, all age bins

In [ ]:
%%time
from common.binning import bin_by_age
from common.wisc import FSIQ, PRIMARY_INDICES
from common.paths import RIDGE_RESULTS

results = []
targets = FSIQ + PRIMARY_INDICES

for target in targets:
    y = Y[target]
    bins = bin_by_age(X, y, ages, y)
    bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
    X_all = [X, bin_1[0], bin_2[0], bin_3[0]]
    y_all = [y, bin_1[1], bin_2[1], bin_3[1]]
    bin_labels = ["All  ", "Bin 1", "Bin 2", "Bin 3"]
#     X_all = [X]
#     y_all = [y]
#     bin_labels = ["All  "]
    
    for X_cv, y_cv, bin_label in zip(X_all, y_all, bin_labels):
        # Find best alpha
        estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(1000, 21000, 1000)], 
                                        scoring=regression_scorer, cv=rkf)]
        pipe = make_pipeline(*estimators).fit(X_cv, y_cv)
        ridge_cv = pipe['ridgecv']
        
        # Do permutation test
        estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
        pipe = make_pipeline(*estimators)
        
        score, _, pvalue = permutation_test_score(
            pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_permutations=n_perm, n_jobs=-1)
        results.append({    
            'Model': 'ridge',
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            'Alpha': ridge_cv.alpha_,
            'Score': score,
            'P-value': pvalue,
        })
        print(results[-1])
        
results_df = pd.DataFrame(results)
display(results_df)
results_df.to_csv(RIDGE_RESULTS)

{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'All  ', 'Alpha': 16000, 'Score': 0.37091336391988167, 'P-value': 0.0003332222592469177}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 1', 'Alpha': 1000, 'Score': 0.1735052160991861, 'P-value': 0.07630789736754415}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 2', 'Alpha': 5000, 'Score': 0.34779987507031157, 'P-value': 0.0003332222592469177}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 3', 'Alpha': 1000, 'Score': 0.0789517941358648, 'P-value': 0.242919026991003}
